In [1]:
from langchain_groq import ChatGroq


In [2]:
llm=ChatGroq(
     temperature=0,
    groq_api_key='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',
    model_name= "llama-3.1-70b-versatile",
  
)




In [3]:
response= llm.invoke("the first footballer to win 8 ballandors was....  ")
print(response.content)

The answer is Lionel Messi. He won his 7th Ballon d'Or in 2021, and then his 8th is not correct as he has not won 8 Ballon d'Or awards yet. However, he did win his 7th Ballon d'Or award in 2021.


In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-45029?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)


Apply for Lead Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software EngineerBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does mor

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ###INSTRUCTION:
    The scraped text is from the careers page of a website.
    Your job is to extract the job postings following these keys: role, skills, experience, and description. 
    Return them in JSON format only.
    Do not add any preamble, explanation, or extra information.

    ### VALID JSON FORMAT (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})

print(res.content)


```json
[
  {
    "role": "Software Engineer",
    "skills": ["Python", "Java", "C++"],
    "experience": "2-5 years",
    "description": "Design, develop, and test software applications"
  },
  {
    "role": "Data Scientist",
    "skills": ["Machine Learning", "Data Analysis", "Statistics"],
    "experience": "5-10 years",
    "description": "Analyze and interpret complex data to inform business decisions"
  },
  {
    "role": "Marketing Manager",
    "skills": ["Marketing Strategy", "Brand Management", "Social Media"],
    "experience": "5-10 years",
    "description": "Develop and execute marketing campaigns to drive business growth"
  }
]
```


In [52]:
from langchain_core.output_parsers import JsonOutputParser

json_parser= JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer',
  'skills': ['Python', 'Java', 'C++'],
  'experience': '2-5 years',
  'description': 'Design, develop, and test software applications'},
 {'role': 'Data Scientist',
  'skills': ['Machine Learning', 'Data Analysis', 'Statistics'],
  'experience': '5-10 years',
  'description': 'Analyze and interpret complex data to inform business decisions'},
 {'role': 'Marketing Manager',
  'skills': ['Marketing Strategy', 'Brand Management', 'Social Media'],
  'experience': '5-10 years',
  'description': 'Develop and execute marketing campaigns to drive business growth'}]

In [16]:
type(json_res)

list

In [17]:
import pandas as pd
df = pd.read_csv("my_portfolio (1).csv")

df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [20]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [30]:
result = collection.query(query_texts=["experience in python", "experience in react"], n_results=2).get('metadatas', []) 

print(result)


[[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/python-portfolio'}], [{'links': 'https://example.com/react-portfolio'}, {'links': 'https://example.com/react-native-portfolio'}]]


In [59]:
job =json_res

In [60]:
print(job['skills'])

TypeError: list indices must be integers or slices, not str

In [61]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Azam, a fresh graduate in BSCS with a strong interest in AI and Machine Learning. Your goal is to write a cold email to a company expressing your enthusiasm for a role and showcasing your relevant skills and projects. You have completed projects such as diabetes prediction, solar power forecasting, and car price prediction, which demonstrate your ability to apply AI/ML to solve real-world problems.

In the email, briefly describe your skills (e.g., Python, TensorFlow, Scikit-learn) and highlight your relevant experience, including internships or academic achievements. Tailor the message to the company's industry or specific needs, showing how your skills and projects align with their objectives.

The email should be professional, concise, and focused on starting a conversation about opportunities at the company.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expressed Interest in Data Scientist Role at [Company Name]

Dear Hiring Manager,

I am excited to express my interest in the Data Scientist position at [Company Name], as advertised. With a strong foundation in Computer Science and a passion for Artificial Intelligence and Machine Learning, I am confident that my skills and experience make me an ideal fit for this role.

As a recent graduate in BSCS, I have developed a solid skill set in Python, TensorFlow, and Scikit-learn, which I have applied to various projects that demonstrate my ability to drive business value through data-driven insights. Notably, my projects include predicting diabetes onset, forecasting solar power output, and predicting car prices, all of which showcase my capacity to collect, analyze, and interpret complex data to inform decision-making.

I am particularly drawn to [Company Name] because of its commitment to [aspect of company's mission or values that resonates with you]. My academic achievements a